# 1 SCRAPPING THE WIKIPEDIA WEB PAGE
## importing libraries

In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

In [151]:
from sklearn.cluster import KMeans

import folium # map rendering library


In [2]:
site= "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(site).text

In [3]:
soup = BeautifulSoup(page,"html5lib")

## finding right table by run a for loop in search of neigbourhoods borough

In [4]:
tables = soup.find_all("table")
type(tables)

bs4.element.ResultSet

### create dataframe

In [5]:
df_hood= pd.DataFrame(columns=['Postal Code','Borough','Neighborhood'])

In [6]:
row1=tables[0].tbody.find_all('td')
type(row1)
#row1[4].text.replace('/','and')
#col1=row1.find_all('b')
#col1
# TESTING CELL

bs4.element.ResultSet

### using a for loop through the entire table and appending result to dataframe result in below
    

In [7]:
for col in row1:
    if ("Not assigned") not in col.text:
        ab=col.text
        code= ab[1:4]
        hood=ab.split('(')[1][:-3]
        hood=hood.replace('/',', ')
        borough=ab.split('(')[0][4:]
        df_hood=df_hood.append({'Postal Code':code,'Borough':borough,'Neighborhood':hood}, ignore_index=True)
df_hood.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [8]:
df_hood.shape

(103, 3)

### installing geocoder for the first time

In [ ]:
pip install geocoder

### using the geocoder package to get latitiude and longitude

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
postal_code="M5A"# TESTING CODE WITH PARKWOODS POSTALCODE

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print(latitude,longitude)

### reverting to geodate CSV file

In [ ]:
import os
cwd = os.getcwd()
cwd #FINDING CURRENT DIRECTORY

In [10]:
geodata=pd.read_csv("Geospatial_Coordinates.csv")
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


df_h=df_hood
df_h[['latitude']]=geodata['Latitude'](df_h['Postal Code'] == geodata['Postal Code'])
df_h.head()

##### MERGING TWO DATAFRAMES

In [11]:
df_hood =df_hood.join(geodata.set_index('Postal Code'),on='Postal Code')
df_hood.head(11)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills)Nort,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


### working with Toronto Boroughs first i explore to know how many are in data

In [44]:
listed=pd.DataFrame(columns=['name'])
for borough in df_hood['Borough']:
    if'Toronto'in borough:
        listed=listed.append({'name':borough},ignore_index=True)
    else:
        None 
listed
toronto_area=listed.name.unique()
toronto_area

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'East YorkEast Toronto', 'Central Toronto',
       'Downtown TorontoStn A PO Boxes25 The Esplanade',
       'East TorontoBusiness reply mail Processing Centre969 Eastern'],
      dtype=object)

In [84]:
df_toronto=pd.DataFrame(columns=df_hood.columns)
type(df_toronto)
df_toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude


## filtering out only borough in toronoto to get "toronto areas"

In [85]:
df_toronto=df_hood[df_hood.Borough.isin(toronto_area)].reset_index().drop(["index"],axis=1)

In [86]:
df_toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [87]:
CLIENT_ID = 'F5KSGNKOZRAOFG4ZXMO1ZJPVN1IGALY0PR5HN0ZOYV345LQH' # your Foursquare ID
CLIENT_SECRET = 'SORNULU0JV3EQL5V4BRVZMNPKKLHYNPSCZYAN4N54H42PKQJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


### function to get venues and create Dataframe

In [88]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### calling Function to get Dataframe of Venues in Toronto area

In [89]:
toronto_venues=getNearbyVenues(df_toronto['Neighborhood'],df_toronto['Latitude'],df_toronto['Longitude'])
(toronto_venues.shape)
toronto_venues.head()
                                                                                            

Regent Park ,  Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond ,  Adelaide ,  King
Dufferin ,  Dovercourt Village
The Danforth  East
Harbourfront East ,  Union Station ,  Toronto Islands
Little Portugal ,  Trinity
The Danforth West ,  Riverdale
Toronto Dominion Centre ,  Design Exchange
Brockton ,  Parkdale Village ,  Exhibition Place
India Bazaar ,  The Beaches West
Commerce Court ,  Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park ,  The Junction South
North Toronto West
The Annex ,  North Midtown ,  Yorkville
Parkdale ,  Roncesvalles
Davisville
University of Toronto ,  Harbord
Runnymede ,  Swansea
Moore Park ,  Summerhill East
Kensington Market ,  Chinatown ,  Grange Park
Summerhill West ,  Rathnelly ,  South Hill ,  Forest Hill SE ,  Deer Park
CN Tower ,  King and Spadina ,  Railway Lands ,  Harbourfront West ,  Bathurst Quay ,  South Niagara ,  Island airport


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park , Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [90]:
print(toronto_venues.shape)

(1575, 7)


### we have 1,575 venues 

## creating a analysis dataframe that can easily allow our cluster model cluster the areas; we use "get dummies "

In [137]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = ([toronto_onehot.columns[160]] + list(toronto_onehot.columns[:159]) + list(toronto_onehot.columns[161:]))
toronto_onehot = toronto_onehot[fixed_columns]
#print(toronto_onehot.columns)
#len(fixed_columns)

In [149]:
a=list(toronto_onehot.columns.values)
a.count('Neighborhood')
toronto_onehot.shape

(1575, 233)

#### now group and consolidate each Neighborhood's on average available of each type of venue in it (eg what is the ratio of schools arround to adult clubs) for neigborhood 1

In [193]:
toronto_hood_ratio= toronto_onehot.groupby("Neighborhood").mean().reset_index()
toronto_hood_ratio.iloc[0,1:].sum()# TOTAL SUM OF RATIO~1

0.9999999999999991

In [194]:
toronto_hood_ratio

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower , King and Spadina , Railway Lands ...",0.000000,0.066667,0.066667,0.133333,0.2,0.066667,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00,0.016667,0.000000,0.000000,0.016667,0.000000,0.000000,0.016667
4,Christie,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.012987,0.000000,0.000000,0.000000,0.0,0.000000,0.012987,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.025974
6,"Commerce Court , Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,0.000000,0.00,...,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.029412,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin , Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Create a cluster Model to learn and cluster in at 233 (venues) Dimension space

In [195]:
# set number of clusters
kclusters = 5

toronto_hood_clustering = toronto_hood_ratio.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_hood_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3,
       0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0])

In [196]:
toronto_hood_ratio.insert(0, 'Cluster Labels',kmeans.labels_)
toronto_hood_ratio.head()


,Cluster Labels,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,0,Berczy Park,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.016949,0.0,0.0,0.000000,0.0,0.0,0.000000
1,0,"Brockton , Parkdale Village , Exhibition Place",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,0,"CN Tower , King and Spadina , Railway Lands ...",0.0,0.066667,0.066667,0.133333,0.2,0.066667,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
3,0,Central Bay Street,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.016667,0.0,0.0,0.016667,0.0,0.0,0.016667
4,0,Christie,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000


In [160]:
toronto_clustered=toronto_hood_ratio[['Cluster Labels','Neighborhood']]

In [161]:
toronto_merged=df_toronto
toronto_merged=toronto_merged.join(toronto_hood_ratio.set_index("Neighborhood"),on= "Neighborhood")
toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Adult Boutique,Airport,Airport Food Court,Airport Lounge,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.022222,0.000000,0.022222
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.000000,0.000000
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.25000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.016667,0.000000,0.000000,0.016667,0.000000,0.000000,0.016667
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,M5H,Downtown Toronto,"Richmond , Adelaide , King",43.650571,-79.384568,0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.010753,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669005,-79.442259,0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106,2,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


lastly Visualization

In [170]:
## getting central cordinates be mean
latitude= toronto_merged['Latitude'].mean()
longitude= toronto_merged['Longitude'].mean()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)
map_clusters

In [177]:

# set color scheme for the clusters
colors=['red','blue','yellow','purple','green']

# add markers to the map
for lat, lon, pop, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(pop) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors[cluster],
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Thank you.